# Travelling Salesman Problem

In [1]:
import pandas as pd
import random
import copy
import math

## Simulated annealing

- function for drawing the initial solution

In [2]:
def random_shuffle(len_data):
    startList = list(range(1, len_data+1))
    initialSolution = startList.copy()
    random.shuffle(initialSolution)
    return initialSolution

- a function that finds a new solution depending on the selected neighborhood

In [4]:
def solution(initialSolution, newSolution, neighborhood, list):
    twoRandomList = random.sample(range(len(list)), 2)

    if (neighborhood == "SWAP"):
        newSolution[twoRandomList[0]] = initialSolution[twoRandomList[1]]
        newSolution[twoRandomList[1]] = initialSolution[twoRandomList[0]]
    elif (neighborhood == "INVERSION"):
        if(twoRandomList[0] < twoRandomList[1]):
            newSolution[twoRandomList[0] : twoRandomList[1] + 1] = reversed(newSolution[twoRandomList[0] : twoRandomList[1] + 1])
        else:
            newSolution[twoRandomList[1] : twoRandomList[0] + 1] = reversed(newSolution[twoRandomList[1] : twoRandomList[0] + 1])
    elif (neighborhood == "INSERTION"):
        if(twoRandomList[0] < twoRandomList[1]):
            newSolution.insert(twoRandomList[0]+1, newSolution.pop(twoRandomList[1]))
        else:
            newSolution.insert(twoRandomList[1]+1, newSolution.pop(twoRandomList[0]))
    
    return newSolution

- function summing up the route length

In [12]:
def routeLengthSum(data, solution):
    routeLength = []

    #adding distances between cities to an empty list
    for i in range(0, len(data)-1):
        routeLength.append(data[solution[i]-1][solution[i+1]-1])

    #adding distances between the last and the first city to the list
    routeLength.append(data[solution[0]-1][solution[-1]-1])

    return sum(routeLength)


In [13]:
def simulatedAnnealing(data, alpha, t, trialsPerEpoch, tMin, neighborhood):
    listt = data.values.tolist()
    initialSolution = random_shuffle(len(listt))
    summ = 499999
    sumSum = 500000 #so that we can save the smallest sum that has rolled over while the algorithm is running
    sumList = []
    while t > tMin:
        trial = 0
        while trial < trialsPerEpoch:
            newSolution = copy.deepcopy(initialSolution)
            solution(initialSolution, newSolution, neighborhood, listt)
            
            newSum = routeLengthSum(listt, newSolution)
            oldSum = routeLengthSum(listt, initialSolution)

            #checking whether the new solution is better than the current one
            if(newSum < oldSum):
                initialSolution = newSolution
                summ = newSum
            else:
                if(random.uniform(0, 1) < math.exp(-(newSum - oldSum)/t)):
                    initialSolution = newSolution
                    summ = newSum
            trial += 1
        t = t * alpha
        #checking if the new smallest sum is less than the "by now" smallest sum
        if(sumSum > summ):
            sumSum = summ
            sumList = newSolution
    print("Sum:", summ)
    print("Order of citis:", initialSolution)
    if(sumSum < summ):
        print("\nPossible lower sum:", sumSum)
        print("Order of tasks:", sumList)

In [9]:
data48 = pd.read_excel("Dane_TSP_48.xlsx", index_col=0)
data76 = pd.read_excel("Dane_TSP_76.xlsx", index_col=0)
data127 = pd.read_excel("Dane_TSP_127.xlsx", index_col=0)

- example of operation

In [ ]:
data = [data48, data76, data127]

for i in range(0,len(data)):
    print("======== SET:", i+1, "========")
    n = 0
    while n < 4:
        print("\nREP:", n+1)    
        simulatedAnnealing(data[i], 0.7, 500, 500, 10, "SWAP")
        n += 1